In [1]:
import sys
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from tqdm import tqdm
import pandas as pd
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [2]:
with open('manual_Quantitation_of_Mitral_Regurgitation.txt','r') as f:
    data = f.readlines()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
complete_data = text_splitter.create_documents(data)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[". "],
    keep_separator=False,
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
complete_data = text_splitter.create_documents(data)


In [5]:
complete_data

[Document(page_content='Quantitation of Mitral Regurgitation'),
 Document(page_content='Significant mitral regurgitation (MR) is estimated to afflict >2 million Americans and is anticipated to increase in prevalence as the baby boomer population ages.1 Approximately 10% of people ≥75 years of age have significant MR,1 and these patients have decreased survival regardless of whether MR is caused by a primary leaflet abnormality2 or is secondary to left ventricular (LV) dysfunction.3–7'),
 Document(page_content='The primary clinical tool for evaluation of the mechanism and severity of MR is echocardiography; however, many patients referred to surgical centers for severe MR by echocardiography have only mild or moderate MR on quantitative evaluation.8 Because surgery is only indicated in patients with severe MR,9,10 it is imperative to quantify MR severity accurately'),
 Document(page_content='In 2003, the American Society of Echocardiography (ASE) and the European Association of Echocard

In [6]:
from sentence_transformers import SentenceTransformer

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(f"Model's maximum sequence length: {SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').max_seq_length}")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(complete_data)]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model's maximum sequence length: 256


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 126/126 [00:00<00:00, 7264.86it/s]


In [7]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    complete_data, embedding_model
)

/usr/local/lib/python3.9/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [8]:
# Embed a user query in the same space
user_query = "Tell me about Vena Contract Width"
query_vector = embedding_model.embed_query(user_query)

In [9]:
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
for i in retrieved_docs:
    print(i.page_content+'\n')
    print('-'*50)


Starting retrieval for user_query='Tell me about Vena Contract Width'...
Vena Contracta Width

--------------------------------------------------
The vena contracta is the smallest, highest-velocity region of a flow jet and is typically located at or just downstream from the regurgitant orifice.35 Its width should be measured in a long-axis imaging plane perpendicular to mitral leaflet closure (Figure 2A).36 If the regurgitant orifice is circular, VCW should directly reflect EROA. Unfortunately, the regurgitant orifice in MR often extends along the mitral coaptation line and is more elliptical than round (Figure 2B)

--------------------------------------------------
The 2-chamber view, which is oriented parallel to the line of leaflet coaptation, may show a wide vena contracta even in mild MR.36 In the setting of a fixed orifice, VCW is independent of flow rate and driving pressure.35 However, the ROA in MR is often dynamic, such that VCW may vary during systole or with changes in he

In [10]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context:
{context}
---
Now here is the question you need to answer.

Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [12]:
question = "how to quantify severe MR using EROA, RgV, RgF"
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=question, k=5)
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
print('Retrieved Documents')
for i in retrieved_docs_text:
    print(i+'\n')
    print('-'*50)
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

print('FINAL ANSWER')
# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Retrieved Documents
Topilsky et al22 have shown that this situation may result in a large EROA but small RgV. Therefore, it is prudent to consider the influence of MR duration, particularly when single-frame measurements of MR severity are used. Quantitation of RgV, in addition to EROA, may help resolve such discrepancies. As noted above, EROA is the product of anatomic ROA and the discharge coefficient, which are the first 2 components of Equation 1

--------------------------------------------------
According to this equation, RgV can be measured by volumetric methods (3D echocardiography, CMR, biplane cineangiography) and EROA calculated from RgV divided by VTI (CW Doppler). Alternatively, EROA can be measured by PISA or 3D imaging and RgV calculated from EROA×VTI. Each method has advantages and disadvantages, but the ability to measure both EROA and RgV is helpful in determining MR severity.

--------------------------------------------------
Strong correlations were seen for EROA 

In [20]:
question = "MR jet meets severe criteria"
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=question, k=5)
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
print('Retrieved Documents')
for i in retrieved_docs_text:
    print(i+'\n')
    print('-'*50)
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

print('FINAL ANSWER')
# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Retrieved Documents
However, a central jet that appears very small (area <4.0 cm2 or <10% of LA area) in multiple views without adhering to any adjacent walls is usually mild MR, especially when the mitral apparatus is structurally normal and the LA and LV are not enlarged.11,12 Conversely, a very large jet that penetrates into the pulmonary veins is likely to be severe MR, especially if associated with a flail leaflet, vegetations, or other significant leaflet pathology

--------------------------------------------------
An MR index has been proposed that weighs 6 different indicators of MR severity with a score of 0 to 3 for jet penetration into the LA, PISA radius, CW jet intensity, pulmonary artery pressure, pulmonary venous flow pattern, and LA size.73 A score of ≤1.7 reliably separated mild MR from severe MR; however, considerable overlap was observed between moderate and severe MR, and the index was not validated against an independent reference standard.

----------------------